Only run if in Colab!

In [ ]:
!git clone https://github.com/ratinac-nikola/rbm.git
!cd rbm
%pip install -r requirements.txt

### Initialization

In [3]:
from sklearn.model_selection import train_test_split
import torch
import config

if torch.cuda.is_available():
    print("Using GPU")
    torch.device('cuda')
else:
    print("Using CPU")

In [ ]:
from dataset import Dataset
dataset = Dataset()
trainset, testset = train_test_split(dataset.getDatasetAsTensor(), train_size=0.1)
trainset.shape

In [5]:
from rbm import RBM
rbm = RBM(trainset.shape[1], hidden_units=10,)

In [ ]:
for epoch in range(1, config.epochs + 1):
    i = 1
    for case in trainset:

        goodSample = case
        badSample = case

        for k in range(10):
            _, hk = rbm.sample_h(badSample)  # forward pass
            _, badSample = rbm.sample_v(hk)  # backward pass

        rbm.train(goodSample, badSample)
        i += 1
        if i % 10 == 0:
            print(f"Case {i} done")
    print(f"Epoch {epoch} done.", end=" ")
